# two_party_continual_input_binay_classification

## 資料品質檢查

### 設定資料路徑 & 參數

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "motor_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/motor_hetero_guest.csv")

### 缺失值 & 欄位名

In [2]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().sum())
print(dense_df.head(5))

idx               0
motor_speed       0
pm                0
stator_yoke       0
stator_tooth      0
stator_winding    0
dtype: int64
   idx  motor_speed        pm  stator_yoke  stator_tooth  stator_winding
0    1     0.293536 -0.633105    -0.397535     -0.290147       -0.307720
1    2    -1.222430 -0.150656    -0.777613     -1.226843       -1.321856
2    3    -0.951901  1.136038     0.967011      0.454562        0.110899
3    4     1.503905  1.046629     1.291209      1.302569        1.049111
4    5     0.186439 -0.470173    -0.053767     -0.123047       -0.032705


## 上傳資料

In [3]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2023-06-07 06:01:52.244 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070601519561680

2023-06-07 06:01:52.251 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-06-07 06:01:53.267 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:01:53.268 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-06-07 06:01:54.283 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-07 06:01:55.298 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-07 06:01:56.313 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-07 06:01:57.327 | INFO     

### 建構 Training pipeline 範例

使用 `pipeline` 模塊來構建聯邦學習流程

In [4]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [5]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=guest) \
        .set_roles(guest=guest, host=host)

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [6]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "motor_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "motor_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

In [7]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name='motor_speed', label_type='float', 
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [8]:
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [9]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="regression",
                                         objective_param={"objective": "lse"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [10]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="regression")

上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [11]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-07 06:02:05.912 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070602014971610

2023-06-07 06:02:05.920 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-07 06:02:06.929 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-07 06:02:07.947 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:02:07.948 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-07 06:02:08.966 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-07 06:02:09.982 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-07 06:02:10.997 | INFO     | pipel

2023-06-07 06:02:43.527 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-06-07 06:02:44.546 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-07 06:02:45.574 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-07 06:02:46.598 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-07 06:02:47.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
m2023-06-07 06:02:48.631 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:02:48.633 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component in

2023-06-07 06:03:22.869 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:16
2023-06-07 06:03:23.885 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:17
m2023-06-07 06:03:24.907 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:03:24.909 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-06-07 06:03:25.925 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-06-07 06:03:26.943 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:21
2023-06-07 06:03:27.961 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component 

2023-06-07 06:04:00.325 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-06-07 06:04:01.341 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-06-07 06:04:02.356 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-06-07 06:04:03.371 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-06-07 06:04:04.395 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-06-07 06:04:05.427 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-07 06:04:06.446 | INFO     | pip

2023-06-07 06:04:38.020 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:32
2023-06-07 06:04:39.037 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:33
2023-06-07 06:04:40.053 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:34
2023-06-07 06:04:41.083 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:35
2023-06-07 06:04:42.100 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:36
2023-06-07 06:04:43.115 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:37
2023-06-07 06:04:44.129 | INFO     | pip

...

2023-06-07 06:09:39.480 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:33
2023-06-07 06:09:40.501 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:34
2023-06-07 06:09:41.516 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:35
2023-06-07 06:09:42.532 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:36
2023-06-07 06:09:43.550 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:37
2023-06-07 06:09:44.567 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:38
2023-06-07 06:09:45.582 | INFO     | pip

2023-06-07 06:10:17.133 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:11
2023-06-07 06:10:18.156 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:12
2023-06-07 06:10:19.172 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:13
2023-06-07 06:10:20.189 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:14
2023-06-07 06:10:21.204 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:15
2023-06-07 06:10:22.220 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:16
2023-06-07 06:10:23.246 | INFO     | pip

2023-06-07 06:10:54.767 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:48
2023-06-07 06:10:55.783 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:49
2023-06-07 06:10:56.798 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:50
2023-06-07 06:10:57.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:51
2023-06-07 06:10:58.834 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:52
2023-06-07 06:10:59.850 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:53
2023-06-07 06:11:00.866 | INFO     | pip

2023-06-07 06:11:32.417 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:26
2023-06-07 06:11:33.433 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:27
2023-06-07 06:11:34.448 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:28
2023-06-07 06:11:35.467 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:29
2023-06-07 06:11:36.482 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:30
2023-06-07 06:11:37.498 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:31
2023-06-07 06:11:38.514 | INFO     | pip

2023-06-07 06:12:10.071 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:04
2023-06-07 06:12:11.086 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:05
2023-06-07 06:12:12.102 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:06
2023-06-07 06:12:13.118 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:07
2023-06-07 06:12:14.134 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:08
2023-06-07 06:12:15.159 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:09
2023-06-07 06:12:16.178 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_st

當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [12]:
pipeline.dump("pipeline_saved/pipeline_saved_continual_input_regression.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `intersect_0`, `hetero_secureboost_0`

In [13]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/pipeline_saved_continual_input_regression.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [14]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=guest).component_param(table={"name": "motor_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=host).component_param(table={"name": "motor_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [15]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="regression")

整合所有模塊

In [16]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [17]:
predict_pipeline.predict()

2023-06-07 06:12:26.652 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070612261097640

2023-06-07 06:12:26.660 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-07 06:12:27.669 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-07 06:12:28.684 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:12:28.685 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:02
2023-06-07 06:12:29.702 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:03
2023-06-07 06:12:30.717 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:04
2023-06-07 06:12:31.740 | INFO     | pipel

2023-06-07 06:13:03.728 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-06-07 06:13:04.746 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-07 06:13:05.761 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-07 06:13:06.778 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-07 06:13:07.794 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
m2023-06-07 06:13:08.809 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:13:08.810 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component in

2023-06-07 06:13:42.544 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:15
m2023-06-07 06:13:44.595 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:13:44.597 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-06-07 06:13:45.614 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-06-07 06:13:46.630 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-06-07 06:13:47.644 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-06-07 06:13:48.663 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running c

2023-06-07 06:14:20.554 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:53
2023-06-07 06:14:21.576 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-06-07 06:14:22.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-06-07 06:14:23.630 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-06-07 06:14:24.645 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-06-07 06:14:25.661 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-07 06:14:26.675 | INFO     | pip

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [18]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/two_party_continual_input_regression.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/two_party_continual_input_regression.json 
 {
    "hetero_secureboost_0": {
        "train": {
            "explained_variance": 0.9226178534666176,
            "mean_absolute_error": 0.21740692326486935,
            "mean_squared_error": 0.07876890259778647,
            "median_absolute_error": 0.194147640835375,
            "r2_score": 0.922617852701248,
            "root_mean_squared_error": 0.2806579815323029
        }
    }
}


In [20]:
pipeline.get_component('hetero_secureboost_0').get_output_data(10)

,idx,label,predict_result,predict_score,predict_detail,type
0,300,-0.9518769,-0.773935681865375,-0.773935681865375,{'label': -0.773935681865375},train
1,510,0.45712125,0.5033330459946249,0.5033330459946249,{'label': 0.5033330459946249},train
2,511,-1.1899537,-0.9461662783853749,-0.9461662783853749,{'label': -0.9461662783853749},train
3,2,-1.22243,-0.9959933263553751,-0.9959933263553751,{'label': -0.9959933263553751},train
4,432,0.41346514,0.33720430517462496,0.33720430517462496,{'label': 0.33720430517462496},train
5,404,-0.90669906,-0.6531124519253749,-0.6531124519253749,{'label': -0.6531124519253749},train
6,556,-1.0680722,-0.846018224825375,-0.846018224825375,{'label': -0.846018224825375},train
7,211,0.57955635,0.675332198304625,0.675332198304625,{'label': 0.675332198304625},train
8,569,-0.77171844,-0.603088511315375,-0.603088511315375,{'label': -0.603088511315375},train


In [30]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "breast_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/breast_hetero_guest.csv")

In [32]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-07 07:26:29.651 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070726295045270

2023-06-07 07:26:29.658 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-06-07 07:26:30.673 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 07:26:30.674 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-06-07 07:26:31.689 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-07 07:26:32.704 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-07 07:26:33.720 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-07 07:26:34.735 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-07 07:26:35.750 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-07 